In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from cs771 import genSyntheticData as gsd
from cs771 import plotData as pd
from cs771 import optLib as opt
from matplotlib import pyplot as plt
import random

In [7]:
def solver( X, y, timeout, spacing ):
	(n, d) = X.shape
	t = 0
	totTime = 0
	
	# w is the model vector and will get returned once timeout happens
	w = np.zeros( (d,) )
	tic = tm.perf_counter()

In [9]:
traindata = np.loadtxt("train")
y = traindata[:,0]
X = traindata[:,1:]


In [10]:
X.shape

(800, 1000)

In [6]:
while True:
		t = t + 1
		if t % spacing == 0:
			toc = tm.perf_counter()
			totTime = totTime + (toc - tic)
			if totTime > timeout:
				return (w, totTime)
			else:
				tic = tm.perf_counter()
                
return (w, totTime)

SyntaxError: 'return' outside function (<ipython-input-6-4c66e4e3ee3e>, line 7)